In [25]:
import numpy as np

chboard = np.zeros((8,8))
chboard[6,0],chboard[6,1],chboard[6,2],chboard[6,3],chboard[6,4],chboard[6,5],chboard[6,6],chboard[6,7] = 1,2,3,4,5,6,7,8
chboard[7,0],chboard[7,1],chboard[7,2],chboard[7,3],chboard[7,4],chboard[7,5],chboard[7,6],chboard[7,7] = 15,12,13,17,18,14,11,16
chboard[1,0],chboard[1,1],chboard[1,2],chboard[1,3],chboard[1,4],chboard[1,5],chboard[1,6],chboard[1,7] = 21,22,23,24,25,26,27,28
chboard[0,0],chboard[0,1],chboard[0,2],chboard[0,3],chboard[0,4],chboard[0,5],chboard[0,6],chboard[0,7] = 36,31,34,37,38,33,32,35
print(chboard)

# NEED TO CREATE SOMETHING TO PRECHECK THE PIECES FOR SPECIAL CONDITION

# white is player 1, black is player 2
# player 1 perspective: rook,knight,bishop,queen,king,bishop,knight,rook

# pieceid for p1 pawn 01-08, knight 11-12, bishop 13-14, rook 15-16, queen 17, king 18,
# p1 promoted pawn: knight (101 to 109), bishop (111 to 119), rook (121 to 129), queen (131 to 139)

# pieceid for p2 pawn 21-28, knight 31-32, bishop 33-34, rook 35-36, queen 37, king 38,
# p1 promoted pawn: knight (201 to 209), bishop (211 to 219), rook (221 to 229), queen (231 to 239)


[[36. 31. 34. 37. 38. 33. 32. 35.]
 [21. 22. 23. 24. 25. 26. 27. 28.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  2.  3.  4.  5.  6.  7.  8.]
 [15. 12. 13. 17. 18. 14. 11. 16.]]


In [26]:
def uncaplist(chboard,turn): # analyses the board for p1 pieces left, returns a list of p1 pieces left
    p1piecelist = list(range(1,9)) + list(range(11,19)) + list(range(101,109)) + list(range(111,119)) + list(range(121,129)) + list(range(131,139))
    p2piecelist = list(range(21,29)) + list(range(31,39)) + list(range(201,209)) + list(range(211,219)) + list(range(221,229)) + list(range(231,239))
    pieceremain = []
    if turn == 1: # if player 1
        for rank in chboard:
            for file in rank:
                if file in p1piecelist:
                    pieceremain.append(file)
        return pieceremain
    if turn == 2: # if player 2
        for rank in chboard:
            for file in rank:
                if file in p2piecelist:
                    pieceremain.append(file)
        return pieceremain

def piecelocator(chboard,pieceid): # finds location of piece on the board, returns a list [rank,file]
    indexcount = 0
    for rank in chboard:
        for file in rank:
            if file == pieceid:
                y = indexcount // 8
                x = indexcount - ((indexcount // 8) * 8)
                return [y,x]
            indexcount += 1
    return [99,99] # piece do not exist

def p2pawncheckmove(chboard,pieceid,rank,file): # determine if P1pawn can move or capture or NONE, returns True/False
    pawnlist, pawnpos, pawn1list, pawn1mcount = list(range(21,29)), piecelocator(chboard,pieceid), list(range(1,9)), 0 #pawn1mcount = player 1 pawn move-counter
    if pieceid in pawnlist:
        if (0 <= rank <= 7) and (0 <= file <= 7):
            
            #### EN-PASSANT SCENARIO ####
            if len(movehistlist) != 0:
                p1epcaptured = movehistlist[len(movehistlist)-1] # Potential En-Passe Captured pawn moveset
                if p1epcaptured[0] in pawn1list: # condition: Must be player 1 pawn ####
                    for moveset in movehistlist:
                        if moveset[0] == p1epcaptured[0]:
                            pawn1mcount += 1
                    if pawn1mcount == 1: # condition: Must be first move of player 1 pawn ####
                        if p1epcaptured[2] == 4: # condition: Must be two square movement (rank 6 to rank 4)  ####
                            pieceidpos = piecelocator(chboard,pieceid)
                            if ((pieceidpos[1] - p1epcaptured[3]) == 1) or ((pieceidpos[1] - p1epcaptured[3]) == -1): # condition: player 1 pawn must be right beside player 2's pawn
                                if chboard[p1epcaptured[2]+1][p1epcaptured[3]] == 0: # condition: enpasse position must be unoccupied
                                    if ((p1epcaptured[2]+1) == rank) and (p1epcaptured[3] == file): # If rank & file input matches the allowed en-passe coordinates
                                        return [True, 'ep']            
            
            if 0 > (pawnpos[0] + 1) > 7: # Check if pawn can move forward\
                return [False, 'n'] # Cannot move forward anymore, no point determining move or capture
            else:
                if pawnpos[0] == 1:
                    vertf = [[pawnpos[0]+1,pawnpos[1]],[pawnpos[0]+2,pawnpos[1]]]
                else:
                    vertf = [[pawnpos[0]+1,pawnpos[1]],[99,99]]
            if 0 > (pawnpos[1] - 1) > 7: # Check if LDIAG is possible
                print('Error: Invalid move')
                rdiagf = None
            else:
                if pawnpos[0] == 1:
                    rdiagf = [[pawnpos[0]+1,pawnpos[1]+1],[pawnpos[0]+2,pawnpos[1]]]
                else:
                    rdiagf = [[pawnpos[0]+1,pawnpos[1]+1],[99,99]]
            if 0 > (pawnpos[1] + 1) > 7:
                print('Error: Invalid move') # Check if RDIAG is possible
                ldiagf = None
            else:
                if pawnpos[0] == 1:
                    ldiagf = [[pawnpos[0]+1,pawnpos[1]-1],[pawnpos[0]+2,pawnpos[1]]]
                else:
                    ldiagf = [[pawnpos[0]+1,pawnpos[1]-1],[99,99]]
                    
            for i in vertf:
                if i == [rank,file]:
                    if chboard[i[0],i[1]] == 0: # MOVE, NEED 0 TO MOVE
                        return [True, 'n']
            for i in rdiagf:
                if i == [rank,file]:
                    if chboard[i[0],i[1]] != 0: # CAPTURE
                        return [True, 'n']
            for i in ldiagf:
                if i == [rank,file]:
                    if chboard[i[0],i[1]] != 0: # CAPTURE
                        return [True, 'n']
            return [False, 'n'] # vertf,rdiagf,ldiagf do not exist
        else:
            return [False, 'n'] #rank/turn inout out of range
    else:
        return [False, 'n'] #pieceid not in pawnlist

def p1pawncheckmove(chboard,pieceid,rank,file): # determine if P1pawn can move or capture or NONE, returns True/False
    pawnlist, pawnpos, pawn2list, pawn2mcount = list(range(1,9)), piecelocator(chboard,pieceid), list(range(21,29)), 0 #pawn2mcount = player 2 pawn move-counter
    if pieceid in pawnlist:
        if (0 <= rank <= 7) and (0 <= file <= 7): 
    
            #### EN-PASSANT SCENARIO ####
            if len(movehistlist) != 0:
                p2epcaptured = movehistlist[len(movehistlist)-1] # Potential En-Passe Captured pawn moveset
                if p2epcaptured[0] in pawn2list: # condition: Must be player 2 pawn ####
                    for moveset in movehistlist:
                        if moveset[0] == p2epcaptured[0]:
                            pawn2mcount += 1
                    if pawn2mcount == 1: # condition: Must be first move of player 2 pawn ####
                        if p2epcaptured[2] == 3: # condition: Must be two square movement (rank 1 to rank 3)  ####
                            pieceidpos = piecelocator(chboard,pieceid)
                            if ((pieceidpos[1] - p2epcaptured[3]) == 1) or ((pieceidpos[1] - p2epcaptured[3]) == -1): # condition: player 1 pawn must be right beside player 2's pawn
                                if chboard[p2epcaptured[2]-1][p2epcaptured[3]] == 0: # condition: enpasse position must be unoccupied
                                    if ((p2epcaptured[2]-1) == rank) and (p2epcaptured[3] == file): # If rank & file input matches the allowed en-passe coordinates
                                        return [True, 'ep']
                                
            #### MAIN PAWN MOVEMENT/CAPTURE FUNCTIONS ####                    
            if 0 > (pawnpos[0] - 1) > 7: # Check if pawn can move forward
                return [False, 'n'] # Cannot move forward anymore, no point determining move or capture
            else:
                if pawnpos[0] == 6:
                    vertf = [[pawnpos[0]-1,pawnpos[1]],[pawnpos[0]-2,pawnpos[1]]]
                else:
                    vertf = [[pawnpos[0]-1,pawnpos[1]],[99,99]]
            if 0 > (pawnpos[1] - 1) > 7: # Check if LDIAG is possible
                print('Error: Invalid move')
                rdiagf = None
            else:
                if pawnpos[0] == 6:
                    rdiagf = [[pawnpos[0]-1,pawnpos[1]+1],[pawnpos[0]-2,pawnpos[1]]]
                else:
                    rdiagf = [[pawnpos[0]-1,pawnpos[1]+1],[99,99]]
            if 0 > (pawnpos[1] + 1) > 7:
                print('Error: Invalid move') # Check if RDIAG is possible
                ldiagf = None
            else:
                if pawnpos[0] == 6:
                    ldiagf = [[pawnpos[0]-1,pawnpos[1]-1],[pawnpos[0]-2,pawnpos[1]]]
                else:
                    ldiagf = [[pawnpos[0]-1,pawnpos[1]-1],[99,99]]
                    
            for i in vertf:
                if i == [rank,file]:
                    if chboard[i[0],i[1]] == 0: # MOVE  DONT HAVE TO BE 0 TO MOVE
                        return [True, 'n']
            for i in rdiagf:
                if i == [rank,file]:
                    if chboard[i[0],i[1]] != 0: # MOVE
                        return [True, 'n']
            for i in ldiagf:
                if i == [rank,file]:
                    if chboard[i[0],i[1]] != 0: # MOVE
                        return [True, 'n']
            return [False, 'n'] # vertf,rdiagf,ldiagf do not exist
        else:
            return [False, 'n'] #rank/turn inout out of range
    else:
        return [False, 'n'] #pieceid not in pawnlist

def pawncheckmove(chboard,turn,pieceid,rank,file): # CORE PAWN MOVEMENT/CAPTURE FUNCTION, returns [True/False,p/n]
    if turn == 1:
        if (p1pawncheckmove(chboard,pieceid,rank,file)[0]) and (p1pawncheckmove(chboard,pieceid,rank,file)[1] == 'ep'): # EN-PASSANT!
            return [True, 'ep']
        elif p1pawncheckmove(chboard,pieceid,rank,file)[0]:
            if rank == 0: # PROMOTION!
                return [True, 'p']
            else: # 
                return [True, 'n']
        else: 
            return [p1pawncheckmove(chboard,pieceid,rank,file)[0], 'n']
    elif turn == 2:
        if (p2pawncheckmove(chboard,pieceid,rank,file)[0]) and (p2pawncheckmove(chboard,pieceid,rank,file)[1] == 'ep'): # EN-PASSANT!
            return [True, 'ep']
        elif p2pawncheckmove(chboard,pieceid,rank,file)[0]:
            if rank  == 7: # PROMOTION!
                return [True, 'p']
            else: # 
                return [True, 'n']
        else:
            return [p2pawncheckmove(chboard,pieceid,rank,file)[0], 'n']

def pawnpromote(chboard,rank,file,pmtdpieceid): # promotion of pawn piece to the piece of choice, Inputs old (pawn) and new piece ID, Outputs nothing
    chboard[rank,file] = pmtdpieceid
    return

###### NOTE TO SELF ######
# Run a pawncheckmove before applypawnmove to check if piece can promote, if so, ask player which piece to be promoted to

def applypawnmove(chboard,turn,pieceid,pmtdpieceid,rank,file): # moves pawn piece on chboard to index [rank,file], if promoted, promote
    checkresult = pawncheckmove(chboard,turn,pieceid,rank,file) 
    p2piecelist = list(range(21,29)) + list(range(31,39)) + list(range(201,209)) + list(range(211,219)) + list(range(221,229)) + list(range(231,239))
    p1piecelist = list(range(1,9)) + list(range(11,19)) + list(range(101,109)) + list(range(111,119)) + list(range(121,129)) + list(range(131,139))
    if checkresult[0] == True: # pawnmove is valid, make pawn move
        if checkresult[1] == 'ep': # pawn does en-passant
            if turn == 1:
                for r in range(0,8): 
                        for f in range(0,8):
                            if pieceid == chboard[r,f]:
                                chboard[r,f] = 0 # remove pawn from old position
                chboard[rank,file] = pieceid # put pawn in new position
                chboard[(rank+1),file] = 0 # remove captured pawn
                return chboard
            elif turn == 2:
                for r in range(0,8): 
                        for f in range(0,8):
                            if pieceid == chboard[r,f]:
                                chboard[r,f] = 0 # remove pawn from old position
                chboard[rank,file] = pieceid # put pawn in new position
                chboard[(rank-1),file] = 0 # remove captured pawn
                return chboard
                
        elif checkresult[1] == 'p': # pawn must promote
            if turn == 2:
                if (pmtdpieceid in p2piecelist) and (pmtdpieceid not in uncaplist(chboard,turn)):
                    for r in range(0,8): 
                        for f in range(0,8):
                            if pieceid == chboard[r,f]:
                                chboard[r,f] = 0 # remove pawn from old position
                    chboard[rank,file] = pieceid # put pawn in new position
                    pawnpromote(chboard,rank,file,pmtdpieceid) # promote pawn
                    return chboard
                else:  
                    return chboard
            elif turn == 1:
                if (pmtdpieceid in p1piecelist) and (pmtdpieceid not in uncaplist(chboard,turn)):
                    for r in range(0,8): 
                        for f in range(0,8):
                            if pieceid == chboard[r,f]:
                                chboard[r,f] = 0 # remove pawn from old position
                    chboard[rank,file] = pieceid # put pawn in new position
                    pawnpromote(chboard,rank,file,pmtdpieceid) # promote pawn
                    return chboard
                else:
                    return chboard
            else:
                return chboard
            
        else: # No promotion, just move/capture
            for r in range(0,8): 
                for f in range(0,8):
                    if pieceid == chboard[r,f]:
                        chboard[r,f] = 0 # remove pawn from old position
            chboard[rank,file] = pieceid # put pawn in new position
            return chboard
    else:
        return chboard # Error: Move is invalid

In [1]:
movehistlist = [] # Every turn should have a list containing [chesspiece,x,rank,file]

# RUN CLONEBOARD THEN APPLYMOVE BEFORE MOVEHISTORY !!!

def cloneboard(chboard): # copies chboard onto a tempboard
    tempboard = np.zeros((8,8))
    ranks,files = chboard.shape[0],chboard.shape[1]
    tempboard = np.zeros((ranks,files))
    for rrank in range(ranks):
        for ffile in range(files):
            tempboard[rrank,ffile] = chboard[rrank,ffile]
    return tempboard
        
def movehistory(chboard,tempboard,movehistlist): # Modifies a list by adding new moves
    manalysis = [] # FOR CASTLING and EN PASSANT (-1 pawn piece), both chboard is ALWAYS O!
    captured,moved = 'o',0 # captured to determine any piece captured, moved to determine pieceid moved
    rin,fin = -1,-1 # rank index, file index
    staticboard = np.zeros((8,8))
    staticboard[6,0],staticboard[6,1],staticboard[6,2],staticboard[6,3],staticboard[6,4],staticboard[6,5],staticboard[6,6],staticboard[6,7] = 1,2,3,4,5,6,7,8 
    staticboard[7,0],staticboard[7,1],staticboard[7,2],staticboard[7,3],staticboard[7,4],staticboard[7,5],staticboard[7,6],staticboard[7,7] = 15,12,13,17,18,14,11,16
    staticboard[1,0],staticboard[1,1],staticboard[1,2],staticboard[1,3],staticboard[1,4],staticboard[1,5],staticboard[1,6],staticboard[1,7] = 21,22,23,24,25,26,27,28
    staticboard[0,0],staticboard[0,1],staticboard[0,2],staticboard[0,3],staticboard[0,4],staticboard[0,5],staticboard[0,6],staticboard[0,7] = 36,31,34,37,38,33,32,35
    
    for rank in list(range(0,8)): # chboard is current board after player moved
        for file in list(range(0,8)):
            if chboard[rank,file] != tempboard[rank,file]: # tempboard should always be 1 turn BEHIND chboard!
                manalysis.append([tempboard[rank,file],chboard[rank,file],[rank,file]]) # append [tp,chp,[rank,file]]
    chpiececount,tpiececount = np.count_nonzero(chboard),np.count_nonzero(tempboard)
    if chpiececount != tpiececount: # determine if a piece was captured
        captured = 'x' ######
    else:
        captured = 'o' #####
        
    if len(manalysis) == 4: # CASTLING
        if manalysis[0][2][0] == 7: # When P1 castles
            for i in manalysis: 
                for j in i:
                    if j == [7,7]: # King side castle (right)
                        movehistlist.append([18,captured,'k','c'])
                        return                            
            for i in manalysis:
                for j in i:
                    if j == [7,0]: # Queen side castle (left)
                        movehistlist.append([18,captured,'q','c'])
                        return  
        elif manalysis[0][2][0] == 0: # When p2 castles
            for i in manalysis: 
                for j in i:
                    if j == [0,7]: # King side castle (right)
                        movehistlist.append([38,captured,'k','c'])
                        return                            
            for i in manalysis:
                for j in i:
                    if j == [0,0]: # Queen side castle (left)
                        movehistlist.append([38,captured,'q','c'])
                        return  
                        
    elif len(manalysis) == 3: # EN PASSANT
        if len(uncaplist(tempboard,1)) != len(uncaplist(chboard,1)): # player 1 piece was captured
            for sets in manalysis:
                if sets[1] in list(range(21,29)):
                    moved = sets[1]
                    if (sets[0] == 0) and(moved == sets[1]):
                        rank,file = sets[2][0],sets[2][1]
                        movehistlist.append([moved,captured,rank,file])
                        return
        elif len(uncaplist(tempboard,2)) != len(uncaplist(chboard,2)): # player 2 piece was captured
            for sets in manalysis:
                if sets[1] in list(range(1,9)):
                    moved = sets[1]
                    if (sets[0] == 0) and(moved == sets[1]):
                        rank,file = sets[2][0],sets[2][1]
                        movehistlist.append([moved,captured,rank,file])
                        return
                            
    else: # its a normal move/capture
        if manalysis[0][1] == 0: # if piece moved, chboard will have '0' 
            moved = manalysis[0][0] ##### piece that moved
            rank,file = manalysis[1][2][0],manalysis[1][2][1] ##### rank and file piece moved to
            movehistlist.append([moved,captured,rank,file])
            return 
        elif manalysis[1][1] == 0: # if piece moved, chboard will have '0'
            moved = manalysis[1][0] ##### piece that moved
            rank,file = manalysis[0][2][0],manalysis[0][2][1] ##### rank and file piece moved to
            movehistlist.append([moved,captured,rank,file])
            return 

def pawnsim(chboard,turn,pieceid,pmtdpieceid,rank,file,movehistlist): # test function for ENTIRE PAWN MOVEMENT
    tempboard = cloneboard(chboard)
    chboard = applypawnmove(chboard,turn,pieceid,pmtdpieceid,rank,file)
    for i in range(8):
        for j in range(8):
            if tempboard[i,j] != chboard[i,j]:
                movehistory(chboard,tempboard,movehistlist)
                return print(chboard,'\n','\n',movehistlist)
    return print(chboard,'\n','\n',movehistlist) # Error

In [ ]:
# tempboard shows you where a piece has moved or captured
# chboard shows you initial position of piece

def applymove(chboard,turn,pieceid,rank,file): # MANY CORE FUNCTIONS WILL BE IN APPLYMOVE!!!
    pass